In [ ]:
# !pip3 install torch torchvision torchaudio
# !pip install annoy

In [ ]:
import os
import torch
from torchvision import transforms, models
from PIL import Image
from IPython.display import display
import torch.nn as nn

## Preprocess

In [ ]:

path = "/Volumes/Seagate Bac/projects/image_similarity_data/"
images = [i for i in os.listdir(path) if not i.startswith(".")]

weights = models.ResNet50_Weights.IMAGENET1K_V2
model = models.resnet50(weights=weights)

model.eval()

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

## Validate tags

In [ ]:
n = 5
n = min(n,len(images))
for i in range(n):
    image = Image.open(os.path.join(path,images[i]))
    input_tensor = transform(image).unsqueeze(0)

    if input_tensor.size()[1] == 3:
        output_tensor = model(input_tensor)
        print(f'{images[i]} predicted as {weights.meta["categories"][torch.argmax(output_tensor)]}')
        display(image)


## Update last layer of the model to get the features

In [ ]:
print(model)

In [ ]:
print(model.fc)

In [ ]:
# since we want the feature vector only and not the classes
model.fc = nn.Identity()
model.eval()

In [ ]:
print(model)

## Create annoy index

In [ ]:
from annoy import AnnoyIndex
n_trees = 20
dimensions = 2048
metric = 'angular'

In [ ]:
annoy_index = AnnoyIndex(dimensions,metric)

In [ ]:
n = len(images)
n = min(n,len(images))
for i in range(n):
    image = Image.open(os.path.join(path,images[i]))
    input_tensor = transform(image).unsqueeze(0)

    if input_tensor.size()[1] == 3:
        output_tensor = model(input_tensor)
        annoy_index.add_item(i,output_tensor[0])

        if i%100  == 0:
            print(f"Processed {i} documents")


## Build Annoy index

In [ ]:
annoy_index.build(n_trees)
annoy_index.save('saved_index.ann')